# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
#!pip install pandas

In [5]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [12]:
#df.head()
df.tail()

,Product,Review
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \r\n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


## LLMChain

In [6]:
#!pip install langchain_community

In [77]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [78]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7, )
#0.7 is a good a bit more creative temperature value

In [81]:
prompt = ChatPromptTemplate.from_template( 'please provide a short summary detailing the technical information about {product}'
 
)

In [82]:

chain = LLMChain(llm=llm, prompt=prompt)

In [95]:
product = 'Samsung A52'
chain.run(product)

"The Samsung A52 is a mid-range smartphone that features a 6.5-inch Super AMOLED display with a resolution of 1080 x 2400 pixels. It is powered by a Qualcomm Snapdragon 720G processor and comes with 6GB or 8GB of RAM options. The phone runs on Android 11 with Samsung's One UI 3.1 on top. It has a quad-camera setup on the back, including a 64MP main camera, a 12MP ultra-wide camera, a 5MP macro camera, and a 5MP depth sensor. The front-facing camera is 32MP. The Samsung A52 also has a 4,500mAh battery with 25W fast charging support."

## SimpleSequentialChain

In [96]:
from langchain.chains import SimpleSequentialChain

In [97]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    'technical details of the product {product} in 20 maximum words'
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [98]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    'summarise the review of the product {product} in a short paragraph'
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [99]:


overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [100]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Samsung A52: 6.5-inch Super AMOLED display, Snapdragon 720G processor, 6/8GB RAM, 128/256GB storage, quad-camera setup, 4500mAh battery.
The Samsung A52 is praised for its vibrant 6.5-inch Super AMOLED display and solid performance with the Snapdragon 720G processor and 6/8GB RAM. The 128/256GB storage options provide ample space for storing files and apps. The quad-camera setup captures good quality photos, and the 4500mAh battery offers decent battery life. Overall, the Samsung A52 is a well-rounded mid-range smartphone with a great display and reliable performance.

> Finished chain.


'The Samsung A52 is praised for its vibrant 6.5-inch Super AMOLED display and solid performance with the Snapdragon 720G processor and 6/8GB RAM. The 128/256GB storage options provide ample space for storing files and apps. The quad-camera setup captures good quality photos, and the 4500mAh battery offers decent battery life. Overall, the Samsung A52 is a well-rounded mid-range smartphone with a great display and reliable performance.'

**Repeat the above twice for different products**

## SequentialChain

In [ ]:
from langchain.chains import SequentialChain


In [142]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  'technical details of the product {product} in 20 maximum words'
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key= 'summary'
                    )


In [143]:
second_prompt = ChatPromptTemplate.from_template(
    'write just they keywords of the technical details of the product {product} in a short paragraph'
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key= 'keywords'
                    )


In [144]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    'translate the {summary} into Italian'
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key= 'Italian'
                      )


In [148]:

#prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        'please rewrite the {summary} in a funny way'
)
# Set the output_key to a valid string, e.g., 'funny_summary'

chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='funny_summary'
                         )

In [153]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables= ["product"],
    output_variables=["summary", "keywords", "Italian", "funny_summary"],
    verbose=True
)

In [154]:
review = df.Review[4]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'product': "\xa0I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can't be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand they will only replace once. Anyway, if you have one good luck.",
 'summary': 'The frother falls out of the handle after a few months of regular use, disappointing durability. Customer service limited to one replacement.',
 'keywords': 'product, frother, handle, replacement, quality, disappointment, warranty, coffee, use, durability',
 'Italian': "Il frullatore cade fuori dall'impugnatura dopo alcuni mesi di uso regolare, deludendo per la durabilità. Il servizio clienti è limitato a una sostituzione.",
 'funny_summary': "Well, the frother decided it had enough of being in the handle after a few months of regular use - talk 

**Repeat the above twice for different products or reviews**

## Router Chain

In [155]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [156]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [157]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [158]:
llm = ChatOpenAI(temperature=0)

In [159]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [160]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [161]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [162]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [163]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [164]:
chain.run("What is a stone?")



> Entering new MultiPromptChain chain...
biology: {'input': 'What is a stone?'}
> Finished chain.


"A stone is a naturally occurring solid substance composed of minerals or mineral-like materials. Stones are typically found in the Earth's crust and can vary in size, shape, color, and composition. They are often used for construction, decoration, and as tools in various cultures. Stones can also play a role in ecosystems, providing habitats for organisms and influencing soil composition."

In [165]:
chain.run("what is a")



> Entering new MultiPromptChain chain...
None: {'input': 'what is a'}
> Finished chain.


'A is the first letter of the English alphabet. It is also a vowel and a letter used in many words in the English language.'

In [ ]:
chain.run("Which one is the worst sickness in the universe?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Which disease is considered the most severe in the universe?'}
> Finished chain.


"As a biologist, I must clarify that there is no single disease that can be considered the most severe in the universe. The severity of a disease can vary depending on factors such as the individual's immune system, access to medical care, and the specific strain of the disease. Some diseases, such as Ebola or certain strains of influenza, can be highly lethal and have caused significant outbreaks in the past. However, the severity of a disease is not solely determined by its mortality rate, but also by its impact on the affected individual's quality of life and the potential for widespread transmission. It is important to approach each disease with caution and understanding, and to continue research and prevention efforts to combat the most severe diseases."

**Repeat the above at least once for different inputs and chains executions - Be creative!**